# Model recommendation with lighfm

### Import libraries

In [ ]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [ ]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [ ]:

TEST_PERCENTAGE = 0.20
LEARNING_RATE = 0.1
NUM_EPOCHS = 80
NUM_COMPONENTS = 60
NUM_THREADS = 4
ALPHA_REG_L2 = 1e-3
MAX_SAMPLED = 10
SEED = 42

### Retrieve data

In [ ]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [ ]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=500000)
df_ratings

In [ ]:
df_ratings.drop(columns=["Unnamed: 0"],inplace=True)
df_ratings.info()

In [ ]:
df_ratings.describe()

In [ ]:
dtype_df_items = {
"page" : 'string',
"url" : 'string',
"issued" : 'string',
"modified" : 'string',
"title" : 'string',
"body" : 'string',
"caption" : 'string',
"age_in_days" : 'UInt32',
"age_exp" : 'Float32',
"age_exp_normalized" : 'Float32',
"ageCategories" : 'category'
}

In [ ]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"], dtype=dtype_df_items)
df_news.drop(columns=["Unnamed: 0"],inplace=True)

In [ ]:
df_news.head(3)

In [ ]:
df_news.info()

In [ ]:
df_merged = pd.merge(df_ratings, df_news, left_on='history', right_on='page', how='left')
df_merged.drop(columns=["page"],inplace=True) # 'page' is the same as 'history'

In [ ]:
df_merged.info()

In [ ]:
df_merged.head()

In [ ]:
w_initial_score = 0.85
w_age_norm = 1-w_initial_score
df_merged["score_init"] = df_merged["score"]*w_initial_score + df_merged["age_exp_normalized"]*w_age_norm
df_merged["score_fn"] = df_merged["score_init"].apply(lambda x: np.power(x,0.33))
df_merged["score_norm"] = (df_merged["score_fn"]-df_merged["score_fn"].min())/(df_merged["score_fn"].max()-df_merged["score_fn"].min())
df_merged["score_1_to_5"] = df_merged["score_norm"]*4+1
df_merged["score_1_to_5_int"] = df_merged["score_1_to_5"].round(0).astype("UInt16")
df_merged.describe()

In [ ]:
column_score_to_use = "score_1_to_5"
df_merged = df_merged[["userId","history","userType",column_score_to_use]]
df_merged.rename(columns={column_score_to_use:"score"},inplace=True)

df_merged.rename(columns={"ageCategories" : "historyFreshnessNormalized"},inplace=True)
df_merged.describe()

In [ ]:
# df_merged.head()
df_merged.head()

In [ ]:
df_merged.info()

### Prepare data

Before fitting the LightFM model, we need to create an instance of `Dataset` which holds the interaction matrix.

In [ ]:
dataset = Dataset()

# Get unique values for users, items, and user features
unique_users = df_merged["userId"].unique()
unique_items = df_merged["history"].unique()
unique_user_features = df_merged["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset.fit(
    users=unique_users,
    items=unique_items,
    user_features=unique_user_features  # Register user features
)


In [ ]:
(interactions, weights) = dataset.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_merged.iterrows()
])


In [ ]:
user_features_list = [
    (row.userId, [row.userType])  
    for _, row in df_merged.iterrows()
]

user_features = dataset.build_user_features(user_features_list)


LightLM works slightly differently compared to other packages as it expects the train and test sets to have same dimension. Therefore the conventional train test split will not work.

The package has included the `cross_validation.random_train_test_split` method to split the interaction data and splits it into two disjoint training and test sets. 

However, note that **it does not validate the interactions in the test set to guarantee all items and users have historical interactions in the training set**. Therefore this may result into a partial cold-start problem in the test set.

In [ ]:
# Split train and test sets (80/20 split)
train, test = cross_validation.random_train_test_split(interactions, test_percentage=TEST_PERCENTAGE, random_state=SEED)
train_weights, test_weights  = cross_validation.random_train_test_split(weights, test_percentage=TEST_PERCENTAGE, random_state=SEED)


Double check the size of both the train and test sets.

In [ ]:
print(f"Shape of train interactions: {train.shape}")
print(f"Shape of test interactions: {test.shape}")

In [ ]:
print(f"Shape of train interactions: {train_weights.shape}")
print(f"Shape of test interactions: {test_weights.shape}")

### Fit the LightFM model

In this notebook, the LightFM model will be using the weighted Approximate-Rank Pairwise (WARP) as the loss. Further explanation on the topic can be found [here](https://making.lyst.com/lightfm/docs/examples/warp_loss.html#learning-to-rank-using-the-warp-loss).


In general, it maximises the rank of positive examples by repeatedly sampling negative examples until a rank violation has been located. This approach is recommended when only positive interactions are present.

The LightFM model can be fitted with the following code:

In [ ]:
model = LightFM(no_components=NUM_COMPONENTS,loss="warp",learning_rate=LEARNING_RATE,user_alpha=ALPHA_REG_L2,max_sampled=MAX_SAMPLED,random_state=np.random.RandomState(SEED))  # Weighted Approximate-Rank Pairwise (WARP) loss
model.fit(train, sample_weight=train_weights, epochs=NUM_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)


### Evaluate model

In [ ]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute evaluation metrics
auc_train = auc_score(model, train, user_features=user_features, num_threads=NUM_THREADS).mean()
auc_test = auc_score(model, test, train_interactions=train, user_features=user_features, num_threads=NUM_THREADS).mean()

# Print evaluation results
print(f"AUC test Score: {auc_test:.4f}")
print(f"AUC train Score: {auc_train:.4f}")

In [ ]:
# # Measure how well it did in the Test period
# for metric in [precision_at_k, recall_at_k]:
#     # Get the precision and recall for Train and Test
#     for data, name in [(train, "Train"), (test, "Test ")]:
#         print(f"{name} {metric.__name__}: %.2f" % 
#               metric(model,
#                          data, 
#                          k=10).mean())

### Save pkls to serve model

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [ ]:
item_id_map_reverse = {v: k for k, v in item_id_map.items()}

In [ ]:
interactions_shape = interactions.shape

In [ ]:
import pickle

pickle.dump(model, open('artifacts/lightfm_model.pkl', 'wb'))
pickle.dump(user_id_map, open('artifacts/user_id_map.pkl', 'wb'))
pickle.dump(item_id_map_reverse, open('artifacts/item_id_map_reverse.pkl', 'wb'))
pickle.dump(user_feature_map, open('artifacts/user_feature_map.pkl', 'wb'))
pickle.dump(interactions_shape, open('artifacts/interactions_shape.pkl', 'wb'))

In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))
loaded_interactions_shape = pickle.load(open('artifacts/interactions_shape.pkl', 'rb'))

### Interaction and Weight Matrices

In [ ]:
# print(weights[:5,:5].todense())

### Scored Matrix

In [ ]:
# # Force lightFM to create predictions for all users and all items
# import numpy as np
# n_items = 5
# n_users = 7

# print([np.full((n_items, ), 1), np.full((n_items, ), 2)])
# print(np.concatenate([np.full((n_items, ), 1), np.full((n_items, ), 2)]))
# print(np.arange((n_items)))

In [ ]:
# n_items = 5
# n_users = 7

# scoring_user_ids = np.concatenate([np.full((n_items, ), i) for i in range(n_users)]) # repeat user ID for number of prods 
# scoring_item_ids = np.concatenate([np.arange(n_items) for i in range(n_users)]) # repeat entire range of item IDs x number of user
# print(scoring_user_ids)
# print(scoring_item_ids)

In [ ]:
loaded_n_users, loaded_n_items = loaded_interactions_shape

In [ ]:
items_idx_to_score = np.arange(loaded_n_items)
scores_matrix_dtype = np.int16

scores = np.zeros((loaded_n_users,loaded_n_items),dtype=scores_matrix_dtype)

for user in range(1000):
    scores[user,] = loaded_model.predict(user_ids = user, item_ids = items_idx_to_score)
    if user%1000 == 0:
        print(f"iteration = {user}")

# scores = model.predict(user_ids = scoring_user_ids, 
#                                      item_ids = scoring_item_ids)
# scores = scores.reshape(-1, n_items) # get 1 row per user
recommendations = pd.DataFrame(scores).astype("Int16")
recommendations.shape

# Have a look at the predicted scores for the first 5 users and first 5 items
recommendations.iloc[:5,:5] 

In [57]:
recommendations.iloc[:5,:5] 

,0,1,2,3,4
0,10,13,4,2,-26
1,7,6,9,75,-29
2,1,37,0,88,84
3,15,-47,16,324,-189
4,-8,34,80,-254,673


In [ ]:
recommendations.min().describe()


In [ ]:
recommendations.max().describe()

In [ ]:
recommendations.mean().describe()

### Make predictions to known and unknowm on same recommendation function with pkls

In [43]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [44]:
def sample_recommendation(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:5]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    print("Top 5 recommended items:")

    for x in top_5_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["page"].values[0])


In [45]:
# predict for known user
user_feature_list = ['userType:Logged']
user_hash = '5f5e17781fc2ec0ddcfb2e9356e61c5d3d4b0b3c8fabd20917feb9e807463856'
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

new user feature encountered 'userType:Logged'
Top 5 recommended items:
        d2593c3d-2347-40d9-948c-b6065e8459a9
        f6b5d170-48b9-4f8e-88d4-c84b6668f3bd
        1f32787b-de2b-49be-8c20-ddaeae34cc22
        bf257382-74fb-4392-ad6a-143240e39f81
        f0a78e58-ec7e-494c-9462-fbd6446a9a89


In [46]:
# predict for unknown user
user_feature_list = ['userType:Non-Logged']
user_hash = ''
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

new user feature encountered 'userType:Non-Logged'
Top 5 recommended items:
        d2593c3d-2347-40d9-948c-b6065e8459a9
        f6b5d170-48b9-4f8e-88d4-c84b6668f3bd
        1f32787b-de2b-49be-8c20-ddaeae34cc22
        bf257382-74fb-4392-ad6a-143240e39f81
        f0a78e58-ec7e-494c-9462-fbd6446a9a89


In [53]:
df_merged.loc[0,'userId']

'fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27288434101e7252ef57'

In [54]:
df_merged.loc[0,'userType']

'Non-Logged'

In [55]:
user_feature_list = ['userType:Non-Logged']
user_hash = 'fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27288434101e7252ef57'
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

Top 5 recommended items:
        c22f9540-546c-499c-8c8a-4fa281ed7377
        248ff2c6-fb06-4429-bdae-03f14f8f3378
        01c814ce-12aa-42a3-9eec-a5e2cf9a522b
        fe553b8c-4e9d-4c05-8e80-5fd7ddd7dacc
        a76034ac-8696-4f17-8d72-91d8ae09dd83


In [56]:
user_id_map["fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27288434101e7252ef57"]

0

In [58]:
df_merged.loc[2,'userId']

'17f1083e6079b0f28f7820a6803583d1c1b405c0718b11a18d30b1620f643b23'

In [59]:
df_merged.loc[2,'userType']

'Non-Logged'

In [ ]:
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)